In [2]:
import wandb
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm 
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.options.display.max_rows = 4000
pd.options.display.max_columns = 500
pd.options.display.max_seq_items = None

In [3]:
from tqdm import tqdm
import multiprocessing
from chaos.results import process_run

project_name = "liac/additives-rebuttal"
#if you ran a sweep you can filter results by the sweep_id
sweeps = ['9ekk8kjy']

api = wandb.Api()
runs = api.runs(project_name, filters={"sweep": {'$in': sweeps}})
run_ids = [run.id for run in runs]
project_names = [project_name]*len(run_ids)

num_processes = 18 #int(multiprocessing.cpu_count() / 2)
args = zip(run_ids, project_names)

with multiprocessing.Pool(num_processes) as pool:
    results = list(tqdm(pool.imap(process_run, args), total=len(run_ids)))



100%|██████████| 115/115 [00:09<00:00, 11.66it/s]


In [4]:
all_history_data = [item[0] for item in results if item is not None]
summary_list = [item[1] for item in results if item[0] is not None]
config_list = [item[2] for item in results if item[0] is not None]
name_list = [item[3] for item in results if item[0] is not None]
run_ids = [item[4] for item in results if item[0] is not None]

history_data = pd.concat(all_history_data)

# Convert to DataFrame
data = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list,
    "run_id": run_ids,
})

# Flatten the dictionaries in the 'summary' and 'config' columns
summary_df = pd.json_normalize(data['summary'])
config_df = pd.json_normalize(data['config'])

# Concatenate all the dataframes together
data = pd.concat([data.drop(columns=['summary', 'config']), summary_df, config_df], axis=1)

In [5]:
# Remove columns that start with "trainer."
data = data.loc[:, ~data.columns.str.startswith('trainer.')]
data = data.loc[:, ~data.columns.str.startswith('model.model_config.')]
data = data.loc[:, ~data.columns.str.startswith('model_config.')]
data = data.loc[:, ~data.columns.str.startswith('model_config/')]

# Remove columns where all values are NaN
data = data.dropna(axis=1, how='all')
data = data.reindex(sorted(data.columns), axis=1)


In [6]:
history_data.shape

(1150, 37)

In [60]:
data.to_csv('../report/summary.csv', index=False)


In [61]:
history_data.to_csv('../report/history.csv', index=False)